# Example: fine-tuning 1B model on CNN Dailymail summarization

In [75]:
!hostname
!pwd

t1v-n-d84a2c81-w-0
/home/patflick/dev/git/maxtext/MaxText


In [12]:
my_bucket = "gs://patflick-maxtext-lingvo"
base_output_directory= my_bucket + "/maxtext/1b/20231120"
checkpoint_path = "gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints"

config_path = "config/base-1b.yml"
# Modify the base config:
extra_config = {
    # Run config
    "run_name": "finetuning-1b-20k-v2",
    "base_output_directory": base_output_directory,
    "save_period": 5000,

    # Load common crawl checkpoint
    "load_from_other_directory": checkpoint_path,
    "load_from_other_directory_step": 3399,

    # Parallelism and KV config
    "dcn_tensor_parallelism": 4,
    "ici_tensor_parallelism": 4,
    "enable_flash_attention": False,

    # use CNN Dailymail summarization dataset
    "dataset_type": "cnndm",

    # Re-use same tokenizer (this currently doesn't work!)
    "assets_path": "/home/patflick/dev/git/maxtext/assets", # TODO: move assets to GCS

    # Config for `training`
    "learning_rate": 3.e-5,
    "steps": 20000,
}

import pyconfig
pyconfig.initialize(["", "configs/base-1b.yml"] + [f"{k}={v}" for k,v in extra_config.items()])
config = pyconfig.config
pyconfig._config.keys

OrderedDict([('run_name', 'finetuning-1b-v1'),
             ('load_parameters_path', ''),
             ('load_from_other_directory',
              'gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints'),
             ('load_from_other_directory_step', 3399),
             ('reuse_example_batch', 0),
             ('metrics_file', ''),
             ('gcs_metrics', False),
             ('dtype', dtype(bfloat16)),
             ('int8_training', False),
             ('use_dqdg', False),
             ('fwd_int8', True),
             ('dlhs_int8', True),
             ('drhs_int8', False),
             ('fwd_int8_qk', False),
             ('dlhs_int8_qk', False),
             ('drhs_int8_qk', False),
             ('fwd_int8_pv', True),
             ('dlhs_int8_pv', True),
             ('drhs_int8_pv', False),
             ('aqt_use_dummy_static_bound', False),
             ('aqt_use_fwd_quant', False),
             ('aqt_rng_type', 'jax.uniform'),
             ('global_parameter_scale', 1),
 

In [13]:
import train
import os
import jax

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
train.train_loop(config) 
jax.distributed.shutdown()

Creating checkpoint manager...


Checkpoint manager created!
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
restoring state from this run's directory latest step         3400
number parameters: 1.091 billion
Per train step, total TFLOPs will be 180.60, split as 89.04% learnable weight flops and 10.96% attention flops
completed step: 3401, seconds: 23.632, TFLOP/s/device: 7.642, loss: 0.064
completed step: 3402, seconds: 0.569, TFLOP/s/device: 317.319, loss: 0.116
completed step: 3403, seconds

In [10]:
# 
import jax
jax.distributed.shutdown()